In [1]:
!pip install selenium

In [2]:
import csv
import requests
import hashlib
import os
import io
from selenium import webdriver
from selenium.common import exceptions
import sys
import time
import selenium

from PIL import Image
import requests
from io import BytesIO

from webdriver_manager.chrome import ChromeDriverManager
#webdriver = webdriver.Chrome(ChromeDriverManager().install())

In [3]:
from distutils.sysconfig import get_python_lib
print(get_python_lib())

C:\Users\Vaidas\anaconda3\Lib\site-packages


In [5]:
def search_and_download(search_term:str,target_path='./images',number_images=20):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(ChromeDriverManager().install()) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)

    '''with concurrent.futures.ThreadPoolExecutor() as head_executor:
        futures_head = [head_executor.submit(persist_image,target_folder, elem)
                   for elem in res]'''
    for elem in res:
        persist_image(target_folder,elem)

In [6]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [7]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(5)
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [9]:
import pandas
zuvys = pandas.read_csv("zuvys.csv")

In [12]:
flist = list(zuvys["zuvis"])
flist

['Lynas',
 'Srovinė aukšlė',
 'Paprastoji aukšlė',
 'Paprastoji saulažuvė',
 'Paprastasis karšis',
 'Paprastasis ūsorius',
 'Paprastasis plakis',
 'Paprastasis',
 'Paprastasis skersasnukis',
 'Paprastasis gružlys',
 'Europinis šapalas',
 'Paprastoji meknė',
 'Paprastasis strepetys',
 'Ežerinė rainė',
 'Paprastoji rainė',
 'Paprastoji kartuolė',
 'Paprastoji raudė',
 'Paprastoji kuoja',
 'Salatis',
 'Šližys',
 'Paprastasis kirtiklis',
 'Šiaurinis auksaspalvis kirtiklis',
 'Paprastasis vijūnas',
 'Paprastasis kūjagalvis',
 'Raibapelekis kūjagalvis',
 'Europinė lydeka',
 'Devynspyglė dyglė',
 'Vėgėlė',
 'Seliava',
 'Ežerinis sykas',
 'Margasis upėtakis',
 'Kiršlys',
 'Paprastasis šamas',
 'Ukraininė nėgė',
 'Mažoji nėgė',
 'Upinė nėgė']

In [ ]:
for zuvis in flist:
    zuvis = zuvis + " zuvis"
    print(zuvis)
    search_and_download(search_term=zuvis,target_path='./images',number_images=200)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


Lynas zuvis




[WDM] - Driver [C:\Users\Vaidas\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


Found: 100 search results. Extracting links from 0:100
Found: 45 image links, looking for more ...
Found: 112 search results. Extracting links from 100:112
Found: 47 image links, looking for more ...
Found: 312 search results. Extracting links from 112:312
Found: 90 image links, looking for more ...
Found: 312 search results. Extracting links from 312:312
Found: 90 image links, looking for more ...


In [13]:
for zuvis in flist:
    print(zuvis)

Lynas
Srovinė aukšlė
Paprastoji aukšlė
Paprastoji saulažuvė
Paprastasis karšis
Paprastasis ūsorius
Paprastasis plakis
Paprastasis
Paprastasis skersasnukis
Paprastasis gružlys
Europinis šapalas
Paprastoji meknė
Paprastasis strepetys
Ežerinė rainė
Paprastoji rainė
Paprastoji kartuolė
Paprastoji raudė
Paprastoji kuoja
Salatis
Šližys
Paprastasis kirtiklis
Šiaurinis auksaspalvis kirtiklis
Paprastasis vijūnas
Paprastasis kūjagalvis
Raibapelekis kūjagalvis
Europinė lydeka
Devynspyglė dyglė
Vėgėlė
Seliava
Ežerinis sykas
Margasis upėtakis
Kiršlys
Paprastasis šamas
Ukraininė nėgė
Mažoji nėgė
Upinė nėgė


In [8]:
search_and_download(search_term='lydeka',target_path='./images',number_images=200)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\Vaidas\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


Found: 100 search results. Extracting links from 0:100
Found: 122 image links, looking for more ...
Found: 312 search results. Extracting links from 100:312
Found: 200 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTGf98BpG4Lj_f5BsFZiuOiVcdI3OD-p_C5ZA&usqp=CAU - as ./images\lydeka\0425acc303.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR7H2FmnzWK-oFOeCwc1JlXNppKKHWMGqTs3g&usqp=CAU - as ./images\lydeka\8f06caf8cf.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSOzfcHfi4zAxUJYj9jL1Bq3LyaofM1mkGQDA&usqp=CAU - as ./images\lydeka\d82bf0232e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS8G05KsRuIG7x-HNKpLboHc78KAW3ukjz8oA&usqp=CAU - as ./images\lydeka\c4f846db72.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS7zHR0rGXVZ99LHdgFm99JavzIvVgvpQf0WA&usqp=CAU - as ./images\lydeka\d8fd847fa4.jpg
SUCCESS - saved https://www.gaspadine.lt/uploads/modu

SUCCESS - saved http://s2.thingpic.com/images/wC/1ZxnP5HBWn3QMfEt5uGe5DCB.jpeg - as ./images\lydeka\892d861838.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRgR0LwXhsVJBeCdhoiseTPyY98rSYKhm9W2g&usqp=CAU - as ./images\lydeka\df78776a08.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRI37leTK_W5iYNcRAriQ7tx7ubobv6AsmvaA&usqp=CAU - as ./images\lydeka\423c558b11.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSsipcfblUhcxhzdZf8frtFxAgf38tu0b6tyw&usqp=CAU - as ./images\lydeka\f7f378826d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSt_obIpL1LqF19UVv6etqaiqGEBvbb15d03g&usqp=CAU - as ./images\lydeka\c18949d21e.jpg
SUCCESS - saved https://lh3.googleusercontent.com/proxy/QqcgqJfnqwt0am8EmbC2LdtLEGuY3R-we9hKTuLb1TzSxAtYNWccCSBZbqe0WY03dxME5zDIdRtAd-cTZu6kysqe_aqtjMS2CIb-1dFG_kqh4Wmak-rMLPtrIHqUIIE - as ./images\lydeka\dfc766e6a5.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images

SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/thumb/b/b3/Esox_masquinongy.jpg/340px-Esox_masquinongy.jpg - as ./images\lydeka\a49c83ae47.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRN791o7NJdFKKgJfBiqwwzlGwF-wDaLMBnHA&usqp=CAU - as ./images\lydeka\9ecbdc82cb.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTSGP2UroYTWfbkHGmdaLQEZSrbm8qAo1Orsw&usqp=CAU - as ./images\lydeka\6e8d637d98.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQSkIA_V47QR3npnOG2dcUJTyU2vMHxwmNEVw&usqp=CAU - as ./images\lydeka\16d5e00c92.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQF7oIlgNkq3Ip10H9Ae2QcPveDyFjXyRtf-A&usqp=CAU - as ./images\lydeka\9f11a8b0b5.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRaDMZb19knOxa1U-tAEWca4lxiRF9DPyItlA&usqp=CAU - as ./images\lydeka\8830bb5d7e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSi2DMuaP1vhW

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQL6xMe9Ub_W6B--kjcQ34Xu5nKi-y7aixb_g&usqp=CAU - as ./images\lydeka\cf74dffe2d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRh4Hj4ftPd95naAiOEnJ0TTKbALuf1Iffk-A&usqp=CAU - as ./images\lydeka\b4ec3a7d8e.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSk6rBtCkCaw8A-cv3NnJW3H-hzXV9oTxOQrg&usqp=CAU - as ./images\lydeka\463383b6d1.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRkwYUxT34OERhbF0S066556lvIE2WjlNVUAA&usqp=CAU - as ./images\lydeka\46b9458539.jpg
SUCCESS - saved https://4.bp.blogspot.com/-6FGXcf86vmY/W-ZIeH0AIWI/AAAAAAAAilc/4LMVJfdzDT0hqar-l9hvv5ABa9byqFrhgCLcBGAs/s1600/krl%2B30%2Bb.jpg - as ./images\lydeka\53f9b723f4.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR_AYAkZBBMmGmthtfn_fu_gEbFWJMeZ1VdIw&usqp=CAU - as ./images\lydeka\7eb0dbd028.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=t